In [ ]:
import pandas as pd
import random
import json
from youtube_dl import YoutubeDL

YCAC_DATASET = 'YCAC_metadata.csv'
YT_DL_DATASET = 'YTDL_metadata.csv'
download_dir = 'yt-music'

In [ ]:
music_data = pd.read_csv(YCAC_DATASET)
music_df = pd.DataFrame(music_data)
music_df.fillna('')
yt_df = pd.DataFrame(data=None, columns=music_df.columns, index=None)

num_samples = 50

In [ ]:
#video = "http://www.youtube.com/watch?v=BaW_jenozKc"
ydl_opts = {
    'outtmpl': download_dir + '/%(title)s.%(ext)s',
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
}
with YoutubeDL(ydl_opts) as ydl:
    for i in range(num_samples):
        try:
            idx = random.randint(0,len(music_df))
            row = music_df.iloc[idx]
            search_string = 'ytsearch1:' + str(row['canonical_title']) + ' ' + str(row['original_composer'])
            info_dict = ydl.extract_info(search_string, download=True)['entries'][0]
            wav_path = download_dir + '/' + info_dict.get('title', None) + '.wav'
            url = info_dict.get('webpage_url')
            #print(info_dict)
            music_df.loc[idx, 'audio_filename'] = wav_path
            music_df.loc[idx, 'webpage_url'] = url
            yt_df = yt_df.append(music_df.iloc[idx])
            print(wav_path)
            print(url)
        except:
            print("An exception occurred.")


yt_df.to_csv(YT_DL_DATASET)    
#yt_df.to_csv(YT_DL_DATASET, mode='a', header=False)
#output = !youtube-dl -x --audio-format 'wav' --print-json -o '{download_dir}/%(title)s.%(ext)s' '{search_string}'
